# Подключения и функции

In [1]:
# Импорт

import pandas as pd
from clickhouse_driver import Client
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Функция создания датафрейма

def create_dataframe(user, password, table_name='films'):
    client = Client('90.156.216.60', user=user, password=password, database='it_minimalist_CH')
    query = f'SELECT * FROM {table_name}'
    df = client.query_dataframe(query)
    return df

В ячейке ниже создается датафрейм в переменной `df`.
<br>

- Замените `'your_username'`, `'your_password'` на ваши учетные данные, которые получили на почту (у каждого своя учетная запись с индивидуальным доступом к БД Clickhouse).
<br>
- Как вы заметили в параметрах функции `create_dataframe`, по умолчанию датафрейм создатеся из таблицы `'films'`. Также в базе данных лежит таблица `'payments'`

In [3]:
films = create_dataframe(user='xxxxx', password='xxxxx', table_name='films')
payments = create_dataframe(user='xxxxx', password='xxxxx', table_name='payments')

In [5]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"


# Семинар 4

Начнем с практики (*вне контекста пандаса*)

In [7]:
def is_cheap(price):
    if price <= 200:
        return 'cheap'
    else:
        return 'expensive'

In [9]:
is_cheap(210)

'expensive'

In [10]:
price = 300
price <= 200

False

In [11]:
# ТАК НЕ НАДО!!!!!!!
def is_cheap(price):
    if price <= 200:
        return True
    else:
        return False

In [13]:
is_cheap(210)

False

In [14]:
# PRO-вариант
def is_cheap(price):
    return price <= 200

In [15]:
is_cheap(210)

False

In [16]:
is_cheap(190)

True

## Лямбда-вариант

In [18]:
if_exp = lambda price: 'cheap' if price <=200 else 'expensive'

In [20]:
if_exp(210)

'expensive'

In [21]:
if_exp(190)

'cheap'

- Лямбда всегда содержит ТОЛЬКО одно выражение, имеет ограниченный спектр применения,
- Нет `return`, само определение функции содержит возвращенное выражение,
- Не нужно присваивать его переменной (но можно),
- Применяется когда в моменте нужна функция одноразово, и нет смысла писать полноценную функцию,
- Часто используется, как аргумент функции высшего порядка (об этом будем говорить отдельно, скоро на основном канале продолжится цикл уроков, там будеи разбирать продвинутый python), так вот лямбды используются, как функция, которая принимает другие функции в качестве аргументов (часто — filter(), map(),reduce(), разберем их.)

### Переходим к пандасу

Когда нам нужно применить какой-то эдакий алгоритм, любой к столбцу пандаса, к *series*, мы используем для этого метод `APPLY`<br>
Принимает на вход **Функцию**, применяется к **Series**

ОТЛИЧАЕТСЯ ТЕМ, ЧТО ПРИМЕНЯЕТСЯ ПОСТРОЧНО (*т.е. к тем объектам, котоыре лежат в ячейках этого столбца (этого series), к которому применяется*

### Пример 1

#### .apply

In [22]:
def is_cheap(price):
    if price <= 200:
        return 'cheap'
    else:
        return 'expensive'

In [23]:
is_cheap(190)

'cheap'

In [26]:
films['is_cheap'] = films.price_ticket.apply(is_cheap)

In [27]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive


In [31]:
films = films.assign(is_cheap_2 = films.price_ticket.apply(lambda price: 'cheap' if price <=200 else 'expensive'))
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap,is_cheap_2
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap,cheap
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap,cheap
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive,expensive


### Пример 2

Хотим сделать все названия фильмов прописными

In [34]:
films.film_name = films.film_name.apply(lambda film_name: film_name.lower())

### Пример 3

ДЛЯ АФИШИ

In [39]:
films['name_length_for_poster'] = films.film_name.apply(lambda film_name: len(film_name))

In [42]:
films.head(3)

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_cheap,is_cheap_2,name_length_for_poster
0,1,побег из шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap,cheap,17
1,2,крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap,cheap,13
2,3,тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive,expensive,13


**ВАЖНО!**

НЕ ИСПОЛЬЗОВАТЬ `apply`, когда можно использовать стандартные методы, способы манипуляции над пандами

## .str (аксессор)
https://pandas.pydata.org/docs/user_guide/text.html

In [16]:
films_short = films.head(8)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма"
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма"
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм"
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма"
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма"


In [21]:
# Определите, начинается ли каждая строка с соответствия регулярному выражению.
films_short['film_name'].str.match('Кр')

0    False
1     True
2    False
3     True
4    False
5    False
6    False
7     True
Name: film_name, dtype: bool

In [34]:
# Считается кол-во подстрок. Например кол-во пробелов
films_short['Кол-во пробелов'] = films_short['film_name'].str.count(' ')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,Кол-во пробелов
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,2
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",1
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",1
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",2
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",2
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",1
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",3
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",1


In [36]:
films_short['film_name'].str.lower()

0                      побег из шоушенка
1                          крёстный отец
2                          тёмный рыцарь
3                        крёстный отец 2
4                 12 разгневанных мужчин
5                        список шиндлера
6    властелин колец: возвращение короля
7                     криминальное чтиво
Name: film_name, dtype: object

In [37]:
films_short['film_name'].str.upper()

0                      ПОБЕГ ИЗ ШОУШЕНКА
1                          КРЁСТНЫЙ ОТЕЦ
2                          ТЁМНЫЙ РЫЦАРЬ
3                        КРЁСТНЫЙ ОТЕЦ 2
4                 12 РАЗГНЕВАННЫХ МУЖЧИН
5                        СПИСОК ШИНДЛЕРА
6    ВЛАСТЕЛИН КОЛЕЦ: ВОЗВРАЩЕНИЕ КОРОЛЯ
7                     КРИМИНАЛЬНОЕ ЧТИВО
Name: film_name, dtype: object

In [38]:
films_short['film_name'].str.len()

0    17
1    13
2    13
3    15
4    22
5    15
6    35
7    18
Name: film_name, dtype: int64

In [47]:
films_short['film_name_mistake'] = films_short['film_name'].apply(lambda x: '  ' + x + ' ')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,Кол-во пробелов,film_name_mistake
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,2,Побег из Шоушенка
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",1,Крёстный отец
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",1,Тёмный рыцарь
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",2,Крёстный отец 2
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",2,12 разгневанных мужчин
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",1,Список Шиндлера
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",3,Властелин колец: Возвращение короля
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",1,Криминальное чтиво


In [48]:
films_short['film_name_mistake'].to_list()

['  Побег из Шоушенка ',
 '  Крёстный отец ',
 '  Тёмный рыцарь ',
 '  Крёстный отец 2 ',
 '  12 разгневанных мужчин ',
 '  Список Шиндлера ',
 '  Властелин колец: Возвращение короля ',
 '  Криминальное чтиво ']

In [50]:
# strip убрать пробелы в начале и в конце
films_short['film_name_correct_all'] = films_short['film_name_mistake'].str.strip()

In [51]:
films_short['film_name_correct_all'].to_list()

['Побег из Шоушенка',
 'Крёстный отец',
 'Тёмный рыцарь',
 'Крёстный отец 2',
 '12 разгневанных мужчин',
 'Список Шиндлера',
 'Властелин колец: Возвращение короля',
 'Криминальное чтиво']

In [52]:
# lstrip убрать пробелы в начале (слева left)
films_short['film_name_correct_left'] = films_short['film_name_mistake'].str.lstrip()

In [53]:
films_short['film_name_correct_left'].to_list()

['Побег из Шоушенка ',
 'Крёстный отец ',
 'Тёмный рыцарь ',
 'Крёстный отец 2 ',
 '12 разгневанных мужчин ',
 'Список Шиндлера ',
 'Властелин колец: Возвращение короля ',
 'Криминальное чтиво ']

In [54]:
# rstrip убрать пробелы в конце (справа right)
films_short['film_name_correct_right'] = films_short['film_name_mistake'].str.rstrip()

In [55]:
films_short['film_name_correct_right'].to_list()

['  Побег из Шоушенка',
 '  Крёстный отец',
 '  Тёмный рыцарь',
 '  Крёстный отец 2',
 '  12 разгневанных мужчин',
 '  Список Шиндлера',
 '  Властелин колец: Возвращение короля',
 '  Криминальное чтиво']

In [68]:
films_short[['kind_film']]

,kind_film
0,драма
1,"детектив, драма"
2,"боевик, детектив, драма"
3,"детектив, драма"
4,"детектив, драма"
5,"байопик, драма, исторический фильм"
6,"боевик, приключение, драма"
7,"детектив, драма"


In [89]:
df_kind_film = films_short['kind_film'].str.split(', ', expand=True)
df_kind_film

,0,1,2
0,драма,None,None
1,детектив,драма,None
2,боевик,детектив,драма
3,детектив,драма,None
4,детектив,драма,None
5,байопик,драма,исторический фильм
6,боевик,приключение,драма
7,детектив,драма,None


In [90]:
df_kind_film.columns = ['Жанр 1', 'Жанр 2', 'Жанр 3']
df_kind_film

,Жанр 1,Жанр 2,Жанр 3
0,драма,None,None
1,детектив,драма,None
2,боевик,детектив,драма
3,детектив,драма,None
4,детектив,драма,None
5,байопик,драма,исторический фильм
6,боевик,приключение,драма
7,детектив,драма,None


In [93]:
df_kind_film['Жанр 1'].str.cat(sep=' | ')

'драма | детектив | боевик | детектив | детектив | байопик | боевик | детектив'

## np.where
<div>
<img src="attachment:021395c0-4d19-4ca6-a17a-956a7b61723d.png" width="500"/>
</div>

In [95]:
import numpy as np

In [96]:
films_short = films.head(8)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма"
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма"
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма"
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма"
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм"
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма"
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма"


In [98]:
films_short['is_exp'] = np.where(films_short['price_ticket'] >= 200, 'expensive', 'cheap')
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_exp
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",expensive
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",expensive
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",cheap
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",expensive
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",expensive


In [101]:
films_short[
(films_short['price_ticket'] >= 200) & 
(films_short['release_year'] >= 2000)
]

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_exp
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",expensive


In [102]:
films_short['сложное условие'] = np.where(
    (films_short['price_ticket'] >= 200) & (films_short['release_year'] >= 2000), 
    1, 
    0
)
films_short

,film_id,film_name,price_ticket,release_year,filmmaker,kind_film,is_exp,сложное условие
0,1,Побег из Шоушенка,190,1994,Фрэнк Дарабонт,драма,cheap,0
1,2,Крёстный отец,190,1972,Фрэнсис Форд Коппола,"детектив, драма",cheap,0
2,3,Тёмный рыцарь,340,2008,Кристофер Нолан,"боевик, детектив, драма",expensive,1
3,4,Крёстный отец 2,290,1974,Фрэнсис Форд Коппола,"детектив, драма",expensive,0
4,5,12 разгневанных мужчин,290,1957,Сидни Люмет,"детектив, драма",expensive,0
5,6,Список Шиндлера,190,1993,Стивен Спилберг,"байопик, драма, исторический фильм",cheap,0
6,7,Властелин колец: Возвращение короля,290,2003,Питер Джексон,"боевик, приключение, драма",expensive,1
7,8,Криминальное чтиво,290,1994,Квентин Тарантино,"детектив, драма",expensive,0


In [109]:
films_short['сложное условие'].mean()

0.25

In [108]:
print('Доля таких фильмов: ' + str(films_short['сложное условие'].mean()*100) + '%')

Доля таких фильмов: 25.0%


# Домашнее задание по семинару 👨‍💻

## 1. Напишите несколько Лямбда-функций: 
1.1 которая принимаюет строку и возвращает ее длину <br>
1.2 которая принимает число и возвращает его квадрат<br>
1.3 которая принимает `float` и возвращает его мантиссу (все, что после плавающей точки). *Тип данных вывода не важен.*

In [1]:
# your code 

## 2. Проверка требований к афише

На основании созданного на семинаре столбца `name_length_for_poster` (для этого надо прогнать весь код выше), создайте столбец `is_fit_to_poster`, в котором будет результат, поместится ли название на афишу (помещается название до 15 символов) <br><br>
Сделайте двумя способами (с помощью обычного логического выражения и с помощью связки **apply + лямбда-функция**)

In [2]:
# your code 

## 3 *. Найти потеряшку

От начальства поступила задача найти конкретного клиента и всю информацию по нему. Однако никто не помнит первый символ в его  `client_id`. Известно, что заканчивается на `G35`.<br><br>

Найдите этого клиента и выведите всю информацию по нему (все строки датафрейма).

*Возможный вариант выполнения:*<br>
*1. Применить связку **apply + lambda** к столбцу `client_id`, вернув срезом только 3 последних символа*<br>
*2. С помощью получившегося **series** с урезанной версией  `client_id` создать логическое выражение (проверить на равенство известной части айдишника)*<br>
*3. С помощью получившегося логического выражения отфильтровать датайрефм, применив логическую индексацию.*<br>

In [3]:
# your code 

## 4. Создайте столбец "Детективчик"

В датафрейме **films** на основе столбца `kind_film` создайте новый столбец с названием "Детективчик". Фильм "Детективчик", если в перечислении жанров в `kind_film` встречается "детектив"

In [ ]:
# your code 

## 5. Что-то такое было на Семинаре 1...

В датафрейме **films** cоздайте столбец при помощи np.where, где проставляется 1, если удовлетворяет условию, и 0, если нет.

**Условие**: все фильмы с ценой либо меньше 190, либо больше 300. Которые снял НЕ Кристофер Нолан в датафрейме `films`

Посчитайте долю фильмов с таким условием

In [ ]:
# your code 